In [81]:
#Steps for today:
#Import the data (V&J)
#Exploring the features (V&J)
#Clean and normalize the data (V&J)
#Decide on a classification algorithm (V&J)
#First trainings (V&J)
#Calculate the scoring and tweak/consolidate features accordingly (V&J)
#First submission

In [82]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import category_encoders as ce
from pathlib import Path  

In [83]:
train_values = pd.read_csv('train_values.csv', sep=',')
train_labels = pd.read_csv('train_labels.csv', sep=',')
x_test = pd.read_csv('test_values.csv', sep=',')


In [84]:
#Encode

ce_te = ce.TargetEncoder(cols=["land_surface_condition","foundation_type","roof_type","ground_floor_type","other_floor_type","position","plan_configuration","legal_ownership_status"])

#columns to perform encoding for train data
X = train_values[["land_surface_condition","foundation_type","roof_type","ground_floor_type","other_floor_type","position","plan_configuration","legal_ownership_status"]]
y = train_values['building_id']

#create an object of the Targetencoder
ce_te.fit(X,y)
encoded_columns = ce_te.transform(X)
normalized_train_values = train_values.copy()

# selecting old value
normalized_train_values[["land_surface_condition","foundation_type","roof_type","ground_floor_type","other_floor_type","position","plan_configuration","legal_ownership_status"]] = encoded_columns[["land_surface_condition","foundation_type","roof_type","ground_floor_type","other_floor_type","position","plan_configuration","legal_ownership_status"]]
new_normalized_train_values= normalized_train_values.drop(columns=['building_id'])

adapted_train_labels = train_labels.drop(columns=['building_id']).values.flatten()


In [85]:
#columns to perform encoding for test data
X_test = x_test[["land_surface_condition","foundation_type","roof_type","ground_floor_type","other_floor_type","position","plan_configuration","legal_ownership_status"]]
y_test = x_test['building_id']

#create an object of the Targetencoder
ce_te.fit(X_test,y_test)
encoded_columns_test = ce_te.transform(X_test)
normalized_train_values_test = x_test.copy()

# selecting old value
normalized_train_values_test[["land_surface_condition","foundation_type","roof_type","ground_floor_type","other_floor_type","position","plan_configuration","legal_ownership_status"]] = encoded_columns_test[["land_surface_condition","foundation_type","roof_type","ground_floor_type","other_floor_type","position","plan_configuration","legal_ownership_status"]]
new_normalized_train_values_test= normalized_train_values_test.drop(columns=['building_id'])
new_normalized_train_values_test

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,17,596,11307,3,20,7,6,527031.605431,527085.741124,526347.561072,...,0,0,0,0,0,0,0,0,0,0
1,6,141,11987,2,25,13,5,527031.605431,527085.741124,526347.561072,...,1,0,0,0,0,0,0,0,0,0
2,22,19,10044,2,5,4,5,527031.605431,527085.741124,526347.561072,...,0,0,0,0,0,0,0,0,0,0
3,26,39,633,1,0,19,3,527031.605431,527085.741124,528794.890677,...,0,0,1,0,0,0,0,0,0,0
4,17,289,7970,3,15,8,7,527031.605431,527085.741124,526894.581632,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86863,4,605,3623,3,70,20,6,527031.605431,527085.741124,526894.581632,...,1,0,0,0,0,0,0,0,0,0
86864,10,1407,11907,3,25,6,7,525246.276988,527085.741124,526347.561072,...,0,0,0,0,0,0,0,0,0,0
86865,22,1136,7712,1,50,3,3,527031.605431,527085.741124,526347.561072,...,0,0,0,0,0,0,0,0,0,0
86866,6,1041,912,2,5,9,5,527031.605431,527085.741124,526347.561072,...,0,0,0,0,0,0,0,0,0,0


In [86]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier()
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(new_normalized_train_values,adapted_train_labels)
y_pred=clf.predict(new_normalized_train_values_test)

i am running
<class 'numpy.ndarray'>


In [89]:
from sklearn.model_selection import cross_val_score

#10-Fold Cross validation
print(np.mean(cross_val_score(clf, new_normalized_train_values, adapted_train_labels)))

0.7196787417430274


In [100]:
#Get submission ready for random forest attempt
import base64

submission_random_forest = x_test["building_id"]
i = pd.DataFrame(np.array(y_pred))

submission_concatenated = pd.concat([submission_random_forest, i], ignore_index="true", axis=1)
submission_renamed_columns = submission_concatenated.rename(columns={0: "building_id", 1: "damage_grade"})
filepath = Path('./submission_random_forest_1.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
submission_renamed_columns.to_csv(filepath, index=False)
